In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, average_precision_score
import numpy as np
import torch.optim as optim


In [2]:
drug_interaction=pd.read_csv("drug_interaction_drugcomb.csv",low_memory=False)
drug_feature=pd.read_csv("drug_feature.csv",index_col=0)
cell_feature=pd.read_csv("final_cell_line_feature.csv",index_col=0)

In [3]:
drug_interaction.shape

(1048575, 18)

In [4]:
drug_interaction.head()

,block_id,drug_row,drug_col,cell_line_name,conc_r_unit,conc_c_unit,css,synergy_zip,synergy_bliss,synergy_loewe,synergy_hsa,ic50_row,ic50_col,ri_row,ri_col,css_row,css_col,S
0,1,5-FU,ABT-888,A2058,uM,uM,30.869,3.865915,6.256584,-2.951386,5.536903,5.126836,3.267734,11.471,-0.441,22.545,39.193,19.839
1,2,5-FU,ABT-888,A2058,uM,uM,27.460,8.247403,12.333896,3.125927,11.614215,5.126836,3.267734,11.471,-0.441,24.135,30.785,16.430
2,3,5-FU,ABT-888,A2058,uM,uM,29.901,6.063440,11.660209,2.452239,10.940528,5.126836,3.267734,11.471,-0.441,25.561,34.241,18.871
3,4,5-FU,ABT-888,A2058,uM,uM,24.016,-4.280231,5.145209,-4.062761,4.425528,5.126836,3.267734,11.471,-0.441,16.661,31.371,12.986
4,5,5-FU,AZD1775,A2058,uM,uM,66.847,12.284698,15.765467,10.409407,18.656340,5.126836,0.266027,11.471,25.164,76.501,57.193,30.212


In [5]:
drug_interaction.dropna(inplace=True)

In [6]:
drug_interaction.shape

(466033, 18)

In [7]:
drug_interaction['synergistic_status'] = drug_interaction['synergy_zip'].apply(lambda x: 1 if x > 0 else 0)

In [8]:
drug_interaction=drug_interaction[['drug_row','drug_col','cell_line_name','synergistic_status']]

In [9]:
drug_interaction.head()

,drug_row,drug_col,cell_line_name,synergistic_status
0,5-FU,ABT-888,A2058,1
1,5-FU,ABT-888,A2058,1
2,5-FU,ABT-888,A2058,1
3,5-FU,ABT-888,A2058,0
4,5-FU,AZD1775,A2058,1


In [10]:
drug_interaction.shape

(466033, 4)

In [11]:
drug_interaction.synergistic_status.value_counts()

synergistic_status
0    258510
1    207523
Name: count, dtype: int64

In [12]:
interaction_drug= set(drug_interaction['drug_row']).union(set(drug_interaction['drug_col']))

In [13]:
len(interaction_drug)

4150

In [14]:
drug_feature.head()

,drug,0,1,2,3,4,5,6,7,8,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,5-FU,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ABT-888,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AZD1775,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,BEZ-235,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,BORTEZOMIB,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
all_drug_with_feature=set(drug_feature['drug'].unique())

In [16]:
len(all_drug_with_feature)

4051

##### if all interaction drug has feature or not

In [17]:
if interaction_drug.issubset(all_drug_with_feature):
    print("All interaction drugs are present in the feature set.")
else:
    print(" Some interaction drugs are missing in the feature set.")
    missing = interaction_drug - all_drug_with_feature
    print("Missing drugs:", missing)


 Some interaction drugs are missing in the feature set.
Missing drugs: {'Curcumin, Curcuma longa L.', 'AK176003', 'AC1Q4OOX', 'METHYL-HESPERIDIN', 'AC1NRBJE', 'GYNOSTEMMA EXTRACT', 'CTK8E8456', 'ZINC242723022', 'AC1OF340', 'AGN-PC-0MU5N5', "Phloretin 2'-galactoside", 'ZINC43163828', 'KS-00001CUZ', 'AC1OFCD4', 'ZINC117147304', 'ZINC34894448', 'ACMC-20mj2l', 'Ergosterine', 'ZINC21999985', 'AC1L34YQ', 'TIC10 isomer', 'AC1L1EUK', 'ZINC11616871', 'AK170455', 'ZINC11616261', 'AC1L1KJK', '1beta-Methylimipenem', 'ZINC18182079', 'ZINC84688828', 'TETRAETHYLENEPENTAMINE 5HCL', 'AC1LAKOZ', 'AC1NPCOI', 'AC1LCZJT', 'AC1L9B3J', 'AK174336', 'AC1OF33F', 'ZINC44963243', 'ZINC3881972', 'AC1OF4NF', 'ZINC38139483', 'ZINC28538988', 'AC1O53DL', 'PLURISIN #1 (NSC 14613)', 'DSSTox_CID_28582', 'ZINC17545571', 'CTK8F0346', 'AC1L1DW2', 'AC1L1IJY', 'ZINC101112865', 'AC1Q29DP', 'QCR-20', 'AC1O7H3F', 'KGBPLKOPSFDBOX-UHFFFAOYSA-N', 'ZINC103939005', '04/04/7218', 'DSSTox_CID_31408', 'Triciribinephosphate', 'AC1M4BZH',

In [18]:
drug_interaction = drug_interaction[
    ~drug_interaction['drug_row'].isin(missing) &
    ~drug_interaction['drug_col'].isin(missing)
].reset_index(drop=True)

In [19]:
drug_interaction.shape

(464744, 4)

In [20]:
all_drugs_before = set(drug_interaction['drug_row']) | set(drug_interaction['drug_col'])
all_drugs_after = set(drug_interaction['drug_row']) | set(drug_interaction['drug_col'])

removed_drugs = all_drugs_before - all_drugs_after
print(f"Number of unique drugs before: {len(all_drugs_before)}")
print(f"Number of unique drugs after: {len(all_drugs_after)}")
print(f"Drugs removed: {len(removed_drugs)}")
print(f"Drugs removed list: {removed_drugs}")

Number of unique drugs before: 4051
Number of unique drugs after: 4051
Drugs removed: 0
Drugs removed list: set()


In [21]:
interaction_drug= set(drug_interaction['drug_row']).union(set(drug_interaction['drug_col']))

In [22]:
interaction_drug_new= set(drug_interaction['drug_row']).union(set(drug_interaction['drug_col']))

In [23]:
len(interaction_drug),len(interaction_drug_new)

(4051, 4051)

In [24]:
all_drug_with_feature==interaction_drug_new

True

In [25]:
drug_feature['drug_id'] = range(len(drug_feature))

In [26]:
drug_to_id = dict(zip(drug_feature['drug'], drug_feature['drug_id']))

In [27]:
drug_feature.drop(columns=['drug_id','drug'],inplace=True)

In [28]:
drug_feature.head()

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
drug_feature.shape

(4051, 2048)

In [30]:
drug_interaction['drug_row_id'] = drug_interaction['drug_row'].map(drug_to_id)
drug_interaction['drug_col_id'] = drug_interaction['drug_col'].map(drug_to_id)

In [31]:
drug_interaction = drug_interaction.drop(columns=['drug_row', 'drug_col'])

In [32]:
cols = ['drug_row_id', 'drug_col_id'] + [col for col in drug_interaction.columns if col not in ['drug_row_id', 'drug_col_id']]
drug_interaction = drug_interaction[cols]

In [33]:
drug_interaction.head()

,drug_row_id,drug_col_id,cell_line_name,synergistic_status
0,0,1,A2058,1
1,0,1,A2058,1
2,0,1,A2058,1
3,0,1,A2058,0
4,0,2,A2058,1


In [34]:
drug_feature.head()

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
cos_sim_matrix = cosine_similarity(drug_feature)

In [35]:
cos_sim_matrix

array([[1.        , 0.17699808, 0.11211037, ..., 0.11646819, 0.09727208,
        0.14808722],
       [0.17699808, 1.        , 0.2073627 , ..., 0.28200837, 0.22898571,
        0.21912525],
       [0.11211037, 0.2073627 , 1.        , ..., 0.17366199, 0.20719896,
        0.34698416],
       ...,
       [0.11646819, 0.28200837, 0.17366199, ..., 1.        , 0.21525296,
        0.27526932],
       [0.09727208, 0.22898571, 0.20719896, ..., 0.21525296, 1.        ,
        0.26000576],
       [0.14808722, 0.21912525, 0.34698416, ..., 0.27526932, 0.26000576,
        1.        ]], shape=(4051, 4051))

In [36]:
cos_sim_matrix.shape

(4051, 4051)

In [37]:
interaction_drug= set(drug_interaction['drug_row_id']).union(set(drug_interaction['drug_col_id']))

In [38]:
len(interaction_drug)

4051

In [39]:
drug_feature.shape

(4051, 2048)

In [40]:
df_synergy = drug_interaction[drug_interaction['synergistic_status'] == 1]


In [41]:
df_synergy

,drug_row_id,drug_col_id,cell_line_name,synergistic_status
0,0,1,A2058,1
1,0,1,A2058,1
2,0,1,A2058,1
4,0,2,A2058,1
5,0,2,A2058,1
...,...,...,...,...
464739,4046,1823,KB-ChR-8-5-11,1
464740,4047,1823,KB-ChR-8-5-11,1
464741,4048,1823,KB-ChR-8-5-11,1
464742,4049,1823,KB-ChR-8-5-11,1


In [42]:
num_drugs = max(drug_interaction['drug_row_id'].max(), drug_interaction['drug_col_id'].max()) + 1


In [43]:
num_drugs

np.int64(4051)

In [44]:
adj_matrix = np.zeros((num_drugs, num_drugs))


In [45]:
adj_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(4051, 4051))

In [46]:
for _, row in df_synergy.iterrows():
    i = int(row['drug_row_id'])
    j = int(row['drug_col_id'])
    adj_matrix[i, j] = 1
    adj_matrix[j, i] = 1  

In [47]:
np.fill_diagonal(adj_matrix, 0)

In [48]:
adj_matrix.shape

(4051, 4051)

In [49]:
adj_matrix

array([[0., 1., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(4051, 4051))

In [50]:
cos_sim_matrix

array([[1.        , 0.17699808, 0.11211037, ..., 0.11646819, 0.09727208,
        0.14808722],
       [0.17699808, 1.        , 0.2073627 , ..., 0.28200837, 0.22898571,
        0.21912525],
       [0.11211037, 0.2073627 , 1.        , ..., 0.17366199, 0.20719896,
        0.34698416],
       ...,
       [0.11646819, 0.28200837, 0.17366199, ..., 1.        , 0.21525296,
        0.27526932],
       [0.09727208, 0.22898571, 0.20719896, ..., 0.21525296, 1.        ,
        0.26000576],
       [0.14808722, 0.21912525, 0.34698416, ..., 0.27526932, 0.26000576,
        1.        ]], shape=(4051, 4051))

In [51]:
drug_feature=np.array(drug_feature)

In [52]:
X = torch.tensor(drug_feature, dtype=torch.float32)        
A_label = torch.tensor(adj_matrix, dtype=torch.float32)
A_cos = torch.tensor(cos_sim_matrix, dtype=torch.float32)

In [53]:
A_label.shape

torch.Size([4051, 4051])

In [54]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X = X.to(device)
A_label = A_label.to(device)



In [55]:
def normalize_adj(adj):
    adj = adj + torch.eye(adj.shape[0]).to(device)  # Add self-loops
    deg = torch.sum(adj, dim=1)
    deg_inv_sqrt = torch.pow(deg, -0.5)
    deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0.0
    D_inv_sqrt = torch.diag(deg_inv_sqrt)
    return D_inv_sqrt @ adj @ D_inv_sqrt

norm_adj = normalize_adj(A_label)

# INITIAL MODEL

In [62]:
class GAE(nn.Module):
    def __init__(self, in_dim, hidden_dim):
        super(GAE, self).__init__()
        self.W = nn.Parameter(torch.randn(in_dim, hidden_dim) * 0.01)

    def encode(self, X, norm_adj):
        return torch.relu(norm_adj @ X @ self.W)  # GCN layer

    def decode(self, Z):
        return torch.sigmoid(Z @ Z.T)  # Inner-product decoder

    def forward(self, X, norm_adj):
        Z = self.encode(X, norm_adj)
        A_pred = self.decode(Z)
        return A_pred, Z

In [63]:
model = GAE(in_dim=2048, hidden_dim=128).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [65]:
triu_indices = torch.triu_indices(X.shape[0], X.shape[0], offset=1)
true_edges = A_label[triu_indices[0], triu_indices[1]]

In [69]:
epochs = 20
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    A_pred, Z = model(X, norm_adj)
    pred_edges = A_pred[triu_indices[0], triu_indices[1]]
    
    loss = criterion(pred_edges, true_edges)
    loss.backward()
    optimizer.step()
    
    # Evaluation metrics
    with torch.no_grad():
        pred_np = pred_edges.detach().cpu().numpy()
        true_np = true_edges.cpu().numpy()
        roc = roc_auc_score(true_np, pred_np)
        pr = average_precision_score(true_np, pred_np)

    print(f"Epoch {epoch+1:03d} | Loss: {loss.item():.4f} | ROC-AUC: {roc:.4f} | PR-AUC: {pr:.4f}")



Epoch 001 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 002 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 003 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 004 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 005 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 006 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 007 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 008 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 009 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 010 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 011 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 012 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 013 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 014 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 015 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 016 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 017 | Loss: 0.6931 | ROC-AUC: 0.50

In [67]:
density = A_label.sum() / (A_label.shape[0] ** 2)
print(f"Sparsity: {density.item():.6f}")


Sparsity: 0.003546


# WITH SELF LOOP

In [70]:
A_label = A_label + torch.eye(A_label.shape[0]).to(device)


In [73]:
def normalize_adj(adj):
    adj = adj + torch.eye(adj.shape[0]).to(device)  # Add self-loops
    deg = torch.sum(adj, dim=1)
    deg_inv_sqrt = torch.pow(deg, -0.5)
    deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0.0
    D_inv_sqrt = torch.diag(deg_inv_sqrt)
    return D_inv_sqrt @ adj @ D_inv_sqrt

norm_adj = normalize_adj(A_label)

In [74]:

true_edges = A_label[triu_indices[0], triu_indices[1]]

In [75]:
epochs = 20
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    A_pred, Z = model(X, norm_adj)
    pred_edges = A_pred[triu_indices[0], triu_indices[1]]
    
    loss = criterion(pred_edges, true_edges)
    loss.backward()
    optimizer.step()
    
    # Evaluation metrics
    with torch.no_grad():
        pred_np = pred_edges.detach().cpu().numpy()
        true_np = true_edges.cpu().numpy()
        roc = roc_auc_score(true_np, pred_np)
        pr = average_precision_score(true_np, pred_np)

    print(f"Epoch {epoch+1:03d} | Loss: {loss.item():.4f} | ROC-AUC: {roc:.4f} | PR-AUC: {pr:.4f}")



Epoch 001 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 002 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 003 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 004 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 005 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 006 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 007 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 008 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 009 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 010 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 011 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 012 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 013 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 014 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 015 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 016 | Loss: 0.6931 | ROC-AUC: 0.5000 | PR-AUC: 0.0035
Epoch 017 | Loss: 0.6931 | ROC-AUC: 0.50

# ReLU TO LeakyReLU IN ENCODER

In [76]:
class GAE(nn.Module):
    def __init__(self, in_dim, hidden_dim):
        super(GAE, self).__init__()
        self.W = nn.Parameter(torch.randn(in_dim, hidden_dim) * 0.01)

    def encode(self, X, norm_adj):
        return F.leaky_relu(norm_adj @ X @ self.W, negative_slope=0.01)


    def decode(self, Z):
        return torch.sigmoid(Z @ Z.T)  # Inner-product decoder

    def forward(self, X, norm_adj):
        Z = self.encode(X, norm_adj)
        A_pred = self.decode(Z)
        return A_pred, Z

In [77]:
model = GAE(in_dim=2048, hidden_dim=128).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [78]:
triu_indices = torch.triu_indices(X.shape[0], X.shape[0], offset=1)
true_edges = A_label[triu_indices[0], triu_indices[1]]

In [79]:
epochs = 20
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    A_pred, Z = model(X, norm_adj)
    pred_edges = A_pred[triu_indices[0], triu_indices[1]]
    
    loss = criterion(pred_edges, true_edges)
    loss.backward()
    optimizer.step()
    
    # Evaluation metrics
    with torch.no_grad():
        pred_np = pred_edges.detach().cpu().numpy()
        true_np = true_edges.cpu().numpy()
        roc = roc_auc_score(true_np, pred_np)
        pr = average_precision_score(true_np, pred_np)

    print(f"Epoch {epoch+1:03d} | Loss: {loss.item():.4f} | ROC-AUC: {roc:.4f} | PR-AUC: {pr:.4f}")



Epoch 001 | Loss: 0.7170 | ROC-AUC: 0.8058 | PR-AUC: 0.0935
Epoch 002 | Loss: 0.6938 | ROC-AUC: 0.9300 | PR-AUC: 0.1327
Epoch 003 | Loss: 0.6949 | ROC-AUC: 0.9298 | PR-AUC: 0.1326
Epoch 004 | Loss: 0.6960 | ROC-AUC: 0.9297 | PR-AUC: 0.1326
Epoch 005 | Loss: 0.6972 | ROC-AUC: 0.9297 | PR-AUC: 0.1326
Epoch 006 | Loss: 0.6983 | ROC-AUC: 0.9296 | PR-AUC: 0.1326
Epoch 007 | Loss: 0.6993 | ROC-AUC: 0.9296 | PR-AUC: 0.1325
Epoch 008 | Loss: 0.7002 | ROC-AUC: 0.9295 | PR-AUC: 0.1325
Epoch 009 | Loss: 0.7009 | ROC-AUC: 0.9295 | PR-AUC: 0.1325
Epoch 010 | Loss: 0.7018 | ROC-AUC: 0.9273 | PR-AUC: 0.1273
Epoch 011 | Loss: 0.7022 | ROC-AUC: 0.9295 | PR-AUC: 0.1325
Epoch 012 | Loss: 0.7027 | ROC-AUC: 0.9290 | PR-AUC: 0.1317
Epoch 013 | Loss: 0.7032 | ROC-AUC: 0.9270 | PR-AUC: 0.1299
Epoch 014 | Loss: 0.7034 | ROC-AUC: 0.9293 | PR-AUC: 0.1320
Epoch 015 | Loss: 0.7036 | ROC-AUC: 0.9293 | PR-AUC: 0.1322
Epoch 016 | Loss: 0.7038 | ROC-AUC: 0.9293 | PR-AUC: 0.1324
Epoch 017 | Loss: 0.7039 | ROC-AUC: 0.92

# USE BCEWithLogitsLoss

In [82]:
class GAE(nn.Module):
    def __init__(self, in_dim, hidden_dim):
        super(GAE, self).__init__()
        self.W = nn.Parameter(torch.randn(in_dim, hidden_dim) * 0.01)

    def encode(self, X, norm_adj):
        return F.leaky_relu(norm_adj @ X @ self.W, negative_slope=0.01)


    def decode(self, Z):
        return  Z @ Z.T  # Inner-product decoder
  
        

    def forward(self, X, norm_adj):
        Z = self.encode(X, norm_adj)
        A_pred = self.decode(Z)
        return A_pred, Z

In [83]:
model = GAE(in_dim=2048, hidden_dim=128).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
pos_weight = (A_label.numel() - A_label.sum()) / A_label.sum()
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

In [84]:
triu_indices = torch.triu_indices(X.shape[0], X.shape[0], offset=1)
true_edges = A_label[triu_indices[0], triu_indices[1]]

In [87]:
epochs = 200
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    A_pred, Z = model(X, norm_adj)
    pred_edges = A_pred[triu_indices[0], triu_indices[1]]
    loss = loss_fn(pred_edges, true_edges)

    
    # loss = criterion(pred_edges, true_edges)
    loss.backward()
    optimizer.step()
    
    # Evaluation metrics
    with torch.no_grad():
        pred_np = pred_edges.detach().cpu().numpy()
        true_np = true_edges.cpu().numpy()
        roc = roc_auc_score(true_np, pred_np)
        pr = average_precision_score(true_np, pred_np)

    print(f"Epoch {epoch+1:03d} | Loss: {loss.item():.4f} | ROC-AUC: {roc:.4f} | PR-AUC: {pr:.4f}")



Epoch 001 | Loss: 1.1430 | ROC-AUC: 0.9329 | PR-AUC: 0.1354
Epoch 002 | Loss: 1.1386 | ROC-AUC: 0.9333 | PR-AUC: 0.1357
Epoch 003 | Loss: 1.1351 | ROC-AUC: 0.9337 | PR-AUC: 0.1360
Epoch 004 | Loss: 1.1324 | ROC-AUC: 0.9341 | PR-AUC: 0.1363
Epoch 005 | Loss: 1.1304 | ROC-AUC: 0.9346 | PR-AUC: 0.1367
Epoch 006 | Loss: 1.1290 | ROC-AUC: 0.9352 | PR-AUC: 0.1371
Epoch 007 | Loss: 1.1280 | ROC-AUC: 0.9358 | PR-AUC: 0.1375
Epoch 008 | Loss: 1.1273 | ROC-AUC: 0.9364 | PR-AUC: 0.1380
Epoch 009 | Loss: 1.1268 | ROC-AUC: 0.9371 | PR-AUC: 0.1385
Epoch 010 | Loss: 1.1265 | ROC-AUC: 0.9378 | PR-AUC: 0.1391
Epoch 011 | Loss: 1.1262 | ROC-AUC: 0.9385 | PR-AUC: 0.1396
Epoch 012 | Loss: 1.1259 | ROC-AUC: 0.9393 | PR-AUC: 0.1403
Epoch 013 | Loss: 1.1255 | ROC-AUC: 0.9401 | PR-AUC: 0.1409
Epoch 014 | Loss: 1.1251 | ROC-AUC: 0.9409 | PR-AUC: 0.1416
Epoch 015 | Loss: 1.1245 | ROC-AUC: 0.9418 | PR-AUC: 0.1423
Epoch 016 | Loss: 1.1237 | ROC-AUC: 0.9426 | PR-AUC: 0.1431
Epoch 017 | Loss: 1.1228 | ROC-AUC: 0.94

In [ ]:
Learnig=ng rate adj

In [ ]:
class GAE(nn.Module):
    def __init__(self, in_dim, hidden_dim):
        super(GAE, self).__init__()
        self.W = nn.Parameter(torch.empty(in_dim, hidden_dim))
        nn.init.xavier_uniform_(self.W)

    def encode(self, X, norm_adj):
        return F.leaky_relu(norm_adj @ X @ self.W, negative_slope=0.01)


    def decode(self, Z):
        return Z @ Z.T  # Inner-product decoder
  
        

    def forward(self, X, norm_adj):
        Z = self.encode(X, norm_adj)
        A_pred = self.decode(Z)
        return A_pred, Z

In [ ]:
model = GAE(in_dim=2048, hidden_dim=128).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
pos_weight = (A_label.numel() - A_label.sum()) / A_label.sum()
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

In [88]:
triu_indices = torch.triu_indices(X.shape[0], X.shape[0], offset=1)
true_edges = A_label[triu_indices[0], triu_indices[1]]

In [89]:
epochs = 200
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    A_pred, Z = model(X, norm_adj)
    pred_edges = A_pred[triu_indices[0], triu_indices[1]]
    loss = loss_fn(pred_edges, true_edges)

    
    # loss = criterion(pred_edges, true_edges)
    loss.backward()
    optimizer.step()
    
    # Evaluation metrics
    with torch.no_grad():
        pred_np = pred_edges.detach().cpu().numpy()
        true_np = true_edges.cpu().numpy()
        roc = roc_auc_score(true_np, pred_np)
        pr = average_precision_score(true_np, pred_np)

    print(f"Epoch {epoch+1:03d} | Loss: {loss.item():.4f} | ROC-AUC: {roc:.4f} | PR-AUC: {pr:.4f}")



Epoch 001 | Loss: 0.9628 | ROC-AUC: 0.9851 | PR-AUC: 0.1859
Epoch 002 | Loss: 0.9624 | ROC-AUC: 0.9851 | PR-AUC: 0.1859
Epoch 003 | Loss: 0.9620 | ROC-AUC: 0.9851 | PR-AUC: 0.1860
Epoch 004 | Loss: 0.9616 | ROC-AUC: 0.9852 | PR-AUC: 0.1861
Epoch 005 | Loss: 0.9612 | ROC-AUC: 0.9852 | PR-AUC: 0.1861
Epoch 006 | Loss: 0.9608 | ROC-AUC: 0.9853 | PR-AUC: 0.1861
Epoch 007 | Loss: 0.9604 | ROC-AUC: 0.9853 | PR-AUC: 0.1861
Epoch 008 | Loss: 0.9600 | ROC-AUC: 0.9853 | PR-AUC: 0.1861
Epoch 009 | Loss: 0.9596 | ROC-AUC: 0.9853 | PR-AUC: 0.1861
Epoch 010 | Loss: 0.9592 | ROC-AUC: 0.9853 | PR-AUC: 0.1861
Epoch 011 | Loss: 0.9588 | ROC-AUC: 0.9854 | PR-AUC: 0.1860
Epoch 012 | Loss: 0.9584 | ROC-AUC: 0.9854 | PR-AUC: 0.1860
Epoch 013 | Loss: 0.9581 | ROC-AUC: 0.9854 | PR-AUC: 0.1860
Epoch 014 | Loss: 0.9577 | ROC-AUC: 0.9854 | PR-AUC: 0.1861
Epoch 015 | Loss: 0.9574 | ROC-AUC: 0.9854 | PR-AUC: 0.1861
Epoch 016 | Loss: 0.9570 | ROC-AUC: 0.9854 | PR-AUC: 0.1860
Epoch 017 | Loss: 0.9567 | ROC-AUC: 0.98

# implementing deep GAE

In [90]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GAE(nn.Module):
    def __init__(self, in_dim, hidden_dim1, hidden_dim2, dropout=0.3):
        super(GAE, self).__init__()
        self.W1 = nn.Parameter(torch.empty(in_dim, hidden_dim1))
        self.W2 = nn.Parameter(torch.empty(hidden_dim1, hidden_dim2))
        
        nn.init.xavier_uniform_(self.W1)
        nn.init.xavier_uniform_(self.W2)

        self.dropout = nn.Dropout(dropout)

    def encode(self, X, norm_adj):
        h1 = F.leaky_relu(norm_adj @ X @ self.W1, negative_slope=0.01)
        h1 = self.dropout(h1)
        h2 = F.leaky_relu(norm_adj @ h1 @ self.W2, negative_slope=0.01)
        return h2  # final node embedding Z

    def decode(self, Z):
        return Z @ Z.T  # Inner-product decoder

    def forward(self, X, norm_adj):
        Z = self.encode(X, norm_adj)
        A_pred = self.decode(Z)
        return A_pred, Z


In [92]:
model = GAE(in_dim=2048, hidden_dim1=512, hidden_dim2=128, dropout=0.3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
pos_weight = (A_label.numel() - A_label.sum()) / A_label.sum()
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

In [93]:
triu_indices = torch.triu_indices(X.shape[0], X.shape[0], offset=1)
true_edges = A_label[triu_indices[0], triu_indices[1]]

In [94]:
epochs = 200
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    A_pred, Z = model(X, norm_adj)
    pred_edges = A_pred[triu_indices[0], triu_indices[1]]
    loss = loss_fn(pred_edges, true_edges)

    
    # loss = criterion(pred_edges, true_edges)
    loss.backward()
    optimizer.step()
    
    # Evaluation metrics
    with torch.no_grad():
        pred_np = pred_edges.detach().cpu().numpy()
        true_np = true_edges.cpu().numpy()
        roc = roc_auc_score(true_np, pred_np)
        pr = average_precision_score(true_np, pred_np)

    print(f"Epoch {epoch+1:03d} | Loss: {loss.item():.4f} | ROC-AUC: {roc:.4f} | PR-AUC: {pr:.4f}")



Epoch 001 | Loss: 1.2613 | ROC-AUC: 0.7227 | PR-AUC: 0.0969
Epoch 002 | Loss: 1.2701 | ROC-AUC: 0.9063 | PR-AUC: 0.1608
Epoch 003 | Loss: 1.1121 | ROC-AUC: 0.9693 | PR-AUC: 0.1769
Epoch 004 | Loss: 1.8424 | ROC-AUC: 0.9725 | PR-AUC: 0.2163
Epoch 005 | Loss: 1.2862 | ROC-AUC: 0.8210 | PR-AUC: 0.0512
Epoch 006 | Loss: 1.2912 | ROC-AUC: 0.9024 | PR-AUC: 0.1483
Epoch 007 | Loss: 1.2052 | ROC-AUC: 0.9571 | PR-AUC: 0.1200
Epoch 008 | Loss: 1.1555 | ROC-AUC: 0.9694 | PR-AUC: 0.1365
Epoch 009 | Loss: 1.2478 | ROC-AUC: 0.9628 | PR-AUC: 0.1970
Epoch 010 | Loss: 1.0783 | ROC-AUC: 0.9737 | PR-AUC: 0.1808
Epoch 011 | Loss: 1.1382 | ROC-AUC: 0.9794 | PR-AUC: 0.2112
Epoch 012 | Loss: 1.0955 | ROC-AUC: 0.9816 | PR-AUC: 0.2112
Epoch 013 | Loss: 1.0426 | ROC-AUC: 0.9813 | PR-AUC: 0.2123
Epoch 014 | Loss: 1.0240 | ROC-AUC: 0.9864 | PR-AUC: 0.2066
Epoch 015 | Loss: 1.0112 | ROC-AUC: 0.9864 | PR-AUC: 0.2285
Epoch 016 | Loss: 1.0216 | ROC-AUC: 0.9874 | PR-AUC: 0.2553
Epoch 017 | Loss: 1.0092 | ROC-AUC: 0.99

# Learnable Bilinear Decoder

In [80]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GAE(nn.Module):
    def __init__(self, in_dim, hidden_dim1, hidden_dim2, dropout=0.3):
        super(GAE, self).__init__()
        self.W1 = nn.Parameter(torch.empty(in_dim, hidden_dim1))
        self.W2 = nn.Parameter(torch.empty(hidden_dim1, hidden_dim2))
        self.decoder_weight = nn.Parameter(torch.empty(hidden_dim2, hidden_dim2))  # ✅ learnable decoder matrix

        nn.init.xavier_uniform_(self.W1)
        nn.init.xavier_uniform_(self.W2)
        nn.init.xavier_uniform_(self.decoder_weight)

        self.dropout = nn.Dropout(dropout)

    def encode(self, X, norm_adj):
        h1 = F.leaky_relu(norm_adj @ X @ self.W1, negative_slope=0.01)
        h1 = self.dropout(h1)
        h2 = F.leaky_relu(norm_adj @ h1 @ self.W2, negative_slope=0.01)
        return h2  

    def decode(self, Z):
        return Z @ self.decoder_weight @ Z.T  # ✅ corrected bilinear decoder

    def forward(self, X, norm_adj):
        Z = self.encode(X, norm_adj)
        A_pred = self.decode(Z)
        return A_pred, Z


In [81]:
model = GAE(in_dim=2048, hidden_dim1=512, hidden_dim2=128, dropout=0.3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
pos_weight = (A_label.numel() - A_label.sum()) / A_label.sum()
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

In [82]:
triu_indices = torch.triu_indices(X.shape[0], X.shape[0], offset=1)
true_edges = A_label[triu_indices[0], triu_indices[1]]

In [83]:
epochs = 200
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    A_pred, Z = model(X, norm_adj)
    pred_edges = A_pred[triu_indices[0], triu_indices[1]]
    loss = loss_fn(pred_edges, true_edges)

    
    # loss = criterion(pred_edges, true_edges)
    loss.backward()
    optimizer.step()
    
    # Evaluation metrics
    with torch.no_grad():
        pred_np = pred_edges.detach().cpu().numpy()
        true_np = true_edges.cpu().numpy()
        roc = roc_auc_score(true_np, pred_np)
        pr = average_precision_score(true_np, pred_np)

    print(f"Epoch {epoch+1:03d} | Loss: {loss.item():.4f} | ROC-AUC: {roc:.4f} | PR-AUC: {pr:.4f}")



Epoch 001 | Loss: 1.3115 | ROC-AUC: 0.8614 | PR-AUC: 0.0585
Epoch 002 | Loss: 75.2828 | ROC-AUC: 0.9339 | PR-AUC: 0.1980
Epoch 003 | Loss: 117.4118 | ROC-AUC: 0.0831 | PR-AUC: 0.0018
Epoch 004 | Loss: 4.5856 | ROC-AUC: 0.0749 | PR-AUC: 0.0018
Epoch 005 | Loss: 1.2375 | ROC-AUC: 0.9478 | PR-AUC: 0.1863
Epoch 006 | Loss: 1.3368 | ROC-AUC: 0.7283 | PR-AUC: 0.1148
Epoch 007 | Loss: 1.3113 | ROC-AUC: 0.9210 | PR-AUC: 0.0825
Epoch 008 | Loss: 1.2148 | ROC-AUC: 0.8998 | PR-AUC: 0.1842
Epoch 009 | Loss: 1.2169 | ROC-AUC: 0.9415 | PR-AUC: 0.1878
Epoch 010 | Loss: 1.3580 | ROC-AUC: 0.8965 | PR-AUC: 0.1861
Epoch 011 | Loss: 1.2606 | ROC-AUC: 0.8726 | PR-AUC: 0.1831
Epoch 012 | Loss: 1.2142 | ROC-AUC: 0.9562 | PR-AUC: 0.1449
Epoch 013 | Loss: 1.2392 | ROC-AUC: 0.9252 | PR-AUC: 0.1176
Epoch 014 | Loss: 1.1651 | ROC-AUC: 0.9571 | PR-AUC: 0.1916
Epoch 015 | Loss: 1.1708 | ROC-AUC: 0.9247 | PR-AUC: 0.1751
Epoch 016 | Loss: 1.1318 | ROC-AUC: 0.9609 | PR-AUC: 0.1916
Epoch 017 | Loss: 1.0941 | ROC-AUC: 0

# FOCAL LOSS 

In [87]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction="mean"):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, logits, targets):
        BCE_loss = F.binary_cross_entropy_with_logits(logits, targets, reduction="none")
        probs = torch.sigmoid(logits)
        pt = probs * targets + (1 - probs) * (1 - targets)
        focal_term = self.alpha * (1 - pt) ** self.gamma
        loss = focal_term * BCE_loss

        if self.reduction == "mean":
            return loss.mean()
        elif self.reduction == "sum":
            return loss.sum()
        return loss

In [88]:
class GAE(nn.Module):
    def __init__(self, in_dim, hidden_dim1, hidden_dim2, dropout=0.3):
        super(GAE, self).__init__()
        self.W1 = nn.Parameter(torch.empty(in_dim, hidden_dim1))
        self.W2 = nn.Parameter(torch.empty(hidden_dim1, hidden_dim2))
        self.decoder_weight = nn.Parameter(torch.empty(hidden_dim2, hidden_dim2))  # learnable decoder matrix

        nn.init.xavier_uniform_(self.W1)
        nn.init.xavier_uniform_(self.W2)
        nn.init.xavier_uniform_(self.decoder_weight)

        self.dropout = nn.Dropout(dropout)

    def encode(self, X, norm_adj):
        h1 = F.leaky_relu(norm_adj @ X @ self.W1, negative_slope=0.01)
        h1 = self.dropout(h1)
        h2 = F.leaky_relu(norm_adj @ h1 @ self.W2, negative_slope=0.01)
        return h2

    def decode(self, Z):
        return Z @ self.decoder_weight @ Z.T

    def forward(self, X, norm_adj):
        Z = self.encode(X, norm_adj)
        A_pred = self.decode(Z)
        return A_pred, Z

In [89]:
model = GAE(in_dim=2048, hidden_dim1=512, hidden_dim2=128, dropout=0.3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

loss_fn = FocalLoss(alpha=0.25, gamma=2.0)

In [90]:
triu_indices = torch.triu_indices(X.shape[0], X.shape[0], offset=1)
true_edges = A_label[triu_indices[0], triu_indices[1]]

In [91]:
epochs = 200
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    A_pred, Z = model(X, norm_adj)
    pred_edges = A_pred[triu_indices[0], triu_indices[1]]
    loss = loss_fn(pred_edges, true_edges)

    loss.backward()
    optimizer.step()

    # Evaluation metrics
    with torch.no_grad():
        pred_np = torch.sigmoid(pred_edges).detach().cpu().numpy()
        true_np = true_edges.cpu().numpy()
        roc = roc_auc_score(true_np, pred_np)
        pr = average_precision_score(true_np, pred_np)

    print(f"Epoch {epoch+1:03d} | Loss: {loss.item():.4f} | ROC-AUC: {roc:.4f} | PR-AUC: {pr:.4f}")

Epoch 001 | Loss: 0.0468 | ROC-AUC: 0.8917 | PR-AUC: 0.0330
Epoch 002 | Loss: 0.2197 | ROC-AUC: 0.0959 | PR-AUC: 0.0030
Epoch 003 | Loss: 0.0263 | ROC-AUC: 0.1711 | PR-AUC: 0.0019
Epoch 004 | Loss: 0.0108 | ROC-AUC: 0.0850 | PR-AUC: 0.0018
Epoch 005 | Loss: 0.0090 | ROC-AUC: 0.0720 | PR-AUC: 0.0018
Epoch 006 | Loss: 0.0101 | ROC-AUC: 0.0781 | PR-AUC: 0.0018
Epoch 007 | Loss: 0.0083 | ROC-AUC: 0.1251 | PR-AUC: 0.0019
Epoch 008 | Loss: 0.0368 | ROC-AUC: 0.3783 | PR-AUC: 0.0074
Epoch 009 | Loss: 0.0202 | ROC-AUC: 0.0753 | PR-AUC: 0.0018
Epoch 010 | Loss: 0.0313 | ROC-AUC: 0.0782 | PR-AUC: 0.0018
Epoch 011 | Loss: 0.0319 | ROC-AUC: 0.0973 | PR-AUC: 0.0019
Epoch 012 | Loss: 0.0253 | ROC-AUC: 0.1361 | PR-AUC: 0.0019
Epoch 013 | Loss: 0.0123 | ROC-AUC: 0.1116 | PR-AUC: 0.0019
Epoch 014 | Loss: 0.0252 | ROC-AUC: 0.7555 | PR-AUC: 0.0421
Epoch 015 | Loss: 0.0135 | ROC-AUC: 0.0511 | PR-AUC: 0.0018
Epoch 016 | Loss: 0.0273 | ROC-AUC: 0.0730 | PR-AUC: 0.0018
Epoch 017 | Loss: 0.0347 | ROC-AUC: 0.08

 # precision-recall curve analysis.

In [95]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GAE(nn.Module):
    def __init__(self, in_dim, hidden_dim1, hidden_dim2, dropout=0.3):
        super(GAE, self).__init__()
        self.W1 = nn.Parameter(torch.empty(in_dim, hidden_dim1))
        self.W2 = nn.Parameter(torch.empty(hidden_dim1, hidden_dim2))
        
        nn.init.xavier_uniform_(self.W1)
        nn.init.xavier_uniform_(self.W2)

        self.dropout = nn.Dropout(dropout)

    def encode(self, X, norm_adj):
        h1 = F.leaky_relu(norm_adj @ X @ self.W1, negative_slope=0.01)
        h1 = self.dropout(h1)
        h2 = F.leaky_relu(norm_adj @ h1 @ self.W2, negative_slope=0.01)
        return h2  # final node embedding Z

    def decode(self, Z):
        return Z @ Z.T  # Inner-product decoder

    def forward(self, X, norm_adj):
        Z = self.encode(X, norm_adj)
        A_pred = self.decode(Z)
        return A_pred, Z


In [96]:
model = GAE(in_dim=2048, hidden_dim1=512, hidden_dim2=128, dropout=0.3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
pos_weight = (A_label.numel() - A_label.sum()) / A_label.sum()
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

In [97]:
triu_indices = torch.triu_indices(X.shape[0], X.shape[0], offset=1)
true_edges = A_label[triu_indices[0], triu_indices[1]]

In [99]:
from sklearn.metrics import precision_recall_curve

epochs = 200
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    A_pred, Z = model(X, norm_adj)
    pred_edges = A_pred[triu_indices[0], triu_indices[1]]
    loss = loss_fn(pred_edges, true_edges)

    
    loss.backward()
    optimizer.step()
    
    
    # Evaluation metrics
    with torch.no_grad():
        pred_np = pred_edges.detach().cpu().numpy()
        true_np = true_edges.cpu().numpy()
    
        # ROC-AUC and PR-AUC
        roc = roc_auc_score(true_np, pred_np)
        pr = average_precision_score(true_np, pred_np)
    
        # Best F1-based threshold from Precision-Recall curve
        precision, recall, thresholds = precision_recall_curve(true_np, pred_np)
        f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
        best_f1_idx = np.argmax(f1)
        best_f1 = f1[best_f1_idx]
        best_threshold = thresholds[best_f1_idx] if best_f1_idx < len(thresholds) else 0.5

    print(f"Epoch {epoch+1:03d} | Loss: {loss.item():.4f} | ROC-AUC: {roc:.4f} | PR-AUC: {pr:.4f} | Best-F1: {best_f1:.4f} @ Thr={best_threshold:.4f}")




Epoch 001 | Loss: 1.5206 | ROC-AUC: 0.9761 | PR-AUC: 0.2473 | Best-F1: 0.3324 @ Thr=4.9222
Epoch 002 | Loss: 1.2223 | ROC-AUC: 0.8856 | PR-AUC: 0.1669 | Best-F1: 0.2993 @ Thr=2.0207
Epoch 003 | Loss: 1.4627 | ROC-AUC: 0.9460 | PR-AUC: 0.1988 | Best-F1: 0.3186 @ Thr=4.9974
Epoch 004 | Loss: 1.3493 | ROC-AUC: 0.7280 | PR-AUC: 0.0177 | Best-F1: 0.0813 @ Thr=0.3216
Epoch 005 | Loss: 1.2862 | ROC-AUC: 0.8984 | PR-AUC: 0.1532 | Best-F1: 0.2907 @ Thr=3.8201
Epoch 006 | Loss: 1.2842 | ROC-AUC: 0.9007 | PR-AUC: 0.1541 | Best-F1: 0.2979 @ Thr=0.5560
Epoch 007 | Loss: 1.2404 | ROC-AUC: 0.9152 | PR-AUC: 0.1884 | Best-F1: 0.3224 @ Thr=0.6833
Epoch 008 | Loss: 1.2068 | ROC-AUC: 0.9153 | PR-AUC: 0.1908 | Best-F1: 0.3193 @ Thr=1.1028
Epoch 009 | Loss: 1.1985 | ROC-AUC: 0.9185 | PR-AUC: 0.1905 | Best-F1: 0.3217 @ Thr=1.7496
Epoch 010 | Loss: 1.2002 | ROC-AUC: 0.9386 | PR-AUC: 0.1938 | Best-F1: 0.3182 @ Thr=2.2924
Epoch 011 | Loss: 1.2349 | ROC-AUC: 0.9340 | PR-AUC: 0.1964 | Best-F1: 0.3215 @ Thr=2.7832

# DROP Edges

In [100]:
class GAE(nn.Module):
    def __init__(self, in_dim, hidden_dim1, hidden_dim2, dropout=0.3):
        super(GAE, self).__init__()
        self.W1 = nn.Parameter(torch.empty(in_dim, hidden_dim1))
        self.W2 = nn.Parameter(torch.empty(hidden_dim1, hidden_dim2))
        self.decoder_weight = nn.Parameter(torch.empty(hidden_dim2, hidden_dim2))
        
        nn.init.xavier_uniform_(self.W1)
        nn.init.xavier_uniform_(self.W2)
        nn.init.xavier_uniform_(self.decoder_weight)

        self.dropout = nn.Dropout(dropout)

    def encode(self, X, norm_adj):
        h1 = F.leaky_relu(norm_adj @ X @ self.W1, negative_slope=0.01)
        h1 = self.dropout(h1)
        h2 = F.leaky_relu(norm_adj @ h1 @ self.W2, negative_slope=0.01)
        return h2

    def decode(self, Z):
        return Z @ self.decoder_weight @ Z.T

    def forward(self, X, norm_adj):
        Z = self.encode(X, norm_adj)
        A_pred = self.decode(Z)
        return A_pred, Z

def drop_edges(adj, drop_prob=0.2):
    mask = torch.rand_like(adj) > drop_prob
    adj_dropped = adj * mask
    return adj_dropped

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [101]:
model = GAE(in_dim=2048, hidden_dim1=512, hidden_dim2=128, dropout=0.3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Compute class imbalance weight for BCE loss
pos_weight = (A_label.numel() - A_label.sum()) / A_label.sum()
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

triu_indices = torch.triu_indices(X.shape[0], X.shape[0], offset=1)
true_edges = A_label[triu_indices[0], triu_indices[1]]

In [102]:
epochs = 200
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    # Drop edges during training
    norm_adj_dropped = drop_edges(norm_adj, drop_prob=0.2)

    A_pred, Z = model(X, norm_adj_dropped)
    pred_edges = A_pred[triu_indices[0], triu_indices[1]]
    loss = loss_fn(pred_edges, true_edges)

    loss.backward()
    optimizer.step()

    with torch.no_grad():
        pred_np = pred_edges.detach().cpu().numpy()
        true_np = true_edges.cpu().numpy()

        roc = roc_auc_score(true_np, pred_np)
        pr = average_precision_score(true_np, pred_np)

        # F1-threshold optimization
        precision, recall, thresholds = precision_recall_curve(true_np, pred_np)
        f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
        best_f1_idx = np.argmax(f1)
        best_f1 = f1[best_f1_idx]
        best_threshold = thresholds[best_f1_idx] if best_f1_idx < len(thresholds) else 0.5

    print(f"Epoch {epoch+1:03d} | Loss: {loss.item():.4f} | ROC-AUC: {roc:.4f} | PR-AUC: {pr:.4f} | Best-F1: {best_f1:.4f} @ Thr={best_threshold:.4f}")

Epoch 001 | Loss: 1.4018 | ROC-AUC: 0.1542 | PR-AUC: 0.0020 | Best-F1: 0.0071 @ Thr=-0.5842
Epoch 002 | Loss: 9.6265 | ROC-AUC: 0.8590 | PR-AUC: 0.1173 | Best-F1: 0.2613 @ Thr=55.1235
Epoch 003 | Loss: 44.8673 | ROC-AUC: 0.1466 | PR-AUC: 0.0019 | Best-F1: 0.0071 @ Thr=-1227.2798
Epoch 004 | Loss: 3.5010 | ROC-AUC: 0.1300 | PR-AUC: 0.0019 | Best-F1: 0.0071 @ Thr=-79.7535
Epoch 005 | Loss: 1.2569 | ROC-AUC: 0.8210 | PR-AUC: 0.1134 | Best-F1: 0.2572 @ Thr=1.0438
Epoch 006 | Loss: 1.3125 | ROC-AUC: 0.8690 | PR-AUC: 0.0640 | Best-F1: 0.1756 @ Thr=0.4551
Epoch 007 | Loss: 1.2368 | ROC-AUC: 0.8772 | PR-AUC: 0.1056 | Best-F1: 0.2418 @ Thr=1.1817
Epoch 008 | Loss: 1.2439 | ROC-AUC: 0.8847 | PR-AUC: 0.1120 | Best-F1: 0.2419 @ Thr=3.4917
Epoch 009 | Loss: 1.2688 | ROC-AUC: 0.8675 | PR-AUC: 0.1060 | Best-F1: 0.2458 @ Thr=3.4762
Epoch 010 | Loss: 1.3701 | ROC-AUC: 0.6035 | PR-AUC: 0.0056 | Best-F1: 0.0232 @ Thr=0.2822
Epoch 011 | Loss: 1.2648 | ROC-AUC: 0.9018 | PR-AUC: 0.1185 | Best-F1: 0.2446 @ T

# L2 regularization (a weight decay style) on the embedding matrix Z

In [103]:
class GAE(nn.Module):
    def __init__(self, in_dim, hidden_dim1, hidden_dim2, dropout=0.3):
        super(GAE, self).__init__()
        self.W1 = nn.Parameter(torch.empty(in_dim, hidden_dim1))
        self.W2 = nn.Parameter(torch.empty(hidden_dim1, hidden_dim2))
        self.decoder_weight = nn.Parameter(torch.empty(hidden_dim2, hidden_dim2))

        nn.init.xavier_uniform_(self.W1)
        nn.init.xavier_uniform_(self.W2)
        nn.init.xavier_uniform_(self.decoder_weight)

        self.dropout = nn.Dropout(dropout)

    def encode(self, X, norm_adj):
        h1 = F.leaky_relu(norm_adj @ X @ self.W1, negative_slope=0.01)
        h1 = self.dropout(h1)
        h2 = F.leaky_relu(norm_adj @ h1 @ self.W2, negative_slope=0.01)
        return h2

    def decode(self, Z):
        return Z @ self.decoder_weight @ Z.T

    def forward(self, X, norm_adj):
        Z = self.encode(X, norm_adj)
        A_pred = self.decode(Z)
        return A_pred, Z

def drop_edges(adj, drop_prob=0.2):
    mask = torch.rand_like(adj) > drop_prob
    adj_dropped = adj * mask
    return adj_dropped

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [104]:
model = GAE(in_dim=2048, hidden_dim1=512, hidden_dim2=128, dropout=0.3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

pos_weight = (A_label.numel() - A_label.sum()) / A_label.sum()
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

triu_indices = torch.triu_indices(X.shape[0], X.shape[0], offset=1)
true_edges = A_label[triu_indices[0], triu_indices[1]]

In [105]:
epochs = 200
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    norm_adj_dropped = drop_edges(norm_adj, drop_prob=0.2)

    A_pred, Z = model(X, norm_adj_dropped)
    pred_edges = A_pred[triu_indices[0], triu_indices[1]]
    
    # Main loss
    loss = loss_fn(pred_edges, true_edges)
    
    # Regularization loss
    reg_loss = torch.norm(Z, p=2)
    loss += 1e-4 * reg_loss  # Small L2 penalty

    loss.backward()
    optimizer.step()

    with torch.no_grad():
        pred_np = pred_edges.detach().cpu().numpy()
        true_np = true_edges.cpu().numpy()

        roc = roc_auc_score(true_np, pred_np)
        pr = average_precision_score(true_np, pred_np)

        precision, recall, thresholds = precision_recall_curve(true_np, pred_np)
        f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
        best_f1_idx = np.argmax(f1)
        best_f1 = f1[best_f1_idx]
        best_threshold = thresholds[best_f1_idx] if best_f1_idx < len(thresholds) else 0.5

    print(f"Epoch {epoch+1:03d} | Loss: {loss.item():.4f} | ROC-AUC: {roc:.4f} | PR-AUC: {pr:.4f} | Best-F1: {best_f1:.4f} @ Thr={best_threshold:.4f}")


Epoch 001 | Loss: 1.3761 | ROC-AUC: 0.6874 | PR-AUC: 0.0100 | Best-F1: 0.0461 @ Thr=0.1117
Epoch 002 | Loss: 17.7411 | ROC-AUC: 0.8752 | PR-AUC: 0.1177 | Best-F1: 0.2545 @ Thr=109.5398
Epoch 003 | Loss: 44.7332 | ROC-AUC: 0.1326 | PR-AUC: 0.0019 | Best-F1: 0.0071 @ Thr=-1244.9591
Epoch 004 | Loss: 1.8636 | ROC-AUC: 0.1757 | PR-AUC: 0.0020 | Best-F1: 0.0071 @ Thr=-23.5822
Epoch 005 | Loss: 1.2439 | ROC-AUC: 0.8565 | PR-AUC: 0.1150 | Best-F1: 0.2560 @ Thr=1.1275
Epoch 006 | Loss: 1.2858 | ROC-AUC: 0.8205 | PR-AUC: 0.0806 | Best-F1: 0.2149 @ Thr=0.8101
Epoch 007 | Loss: 1.2332 | ROC-AUC: 0.8729 | PR-AUC: 0.0834 | Best-F1: 0.2119 @ Thr=1.6370
Epoch 008 | Loss: 1.3696 | ROC-AUC: 0.8465 | PR-AUC: 0.1035 | Best-F1: 0.2425 @ Thr=4.7627
Epoch 009 | Loss: 1.2239 | ROC-AUC: 0.8805 | PR-AUC: 0.0833 | Best-F1: 0.2159 @ Thr=1.6914
Epoch 010 | Loss: 1.1934 | ROC-AUC: 0.9228 | PR-AUC: 0.0943 | Best-F1: 0.2051 @ Thr=1.7167
Epoch 011 | Loss: 1.1688 | ROC-AUC: 0.9405 | PR-AUC: 0.1244 | Best-F1: 0.2433 @ 

In [107]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve
import numpy as np

# Deeper GAE model
class DeepGAE(nn.Module):
    def __init__(self, in_dim, h1, h2, h3, dropout=0.3):
        super(DeepGAE, self).__init__()
        self.W1 = nn.Parameter(torch.empty(in_dim, h1))
        self.W2 = nn.Parameter(torch.empty(h1, h2))
        self.W3 = nn.Parameter(torch.empty(h2, h3))
        self.decoder_weight = nn.Parameter(torch.empty(h3, h3))

        # Xavier initialization
        nn.init.xavier_uniform_(self.W1)
        nn.init.xavier_uniform_(self.W2)
        nn.init.xavier_uniform_(self.W3)
        nn.init.xavier_uniform_(self.decoder_weight)

        self.dropout = nn.Dropout(dropout)

    def encode(self, X, norm_adj):
        h1 = F.leaky_relu(norm_adj @ X @ self.W1, negative_slope=0.01)
        h1 = self.dropout(h1)
        h2 = F.leaky_relu(norm_adj @ h1 @ self.W2, negative_slope=0.01)
        h2 = self.dropout(h2)
        h3 = F.leaky_relu(norm_adj @ h2 @ self.W3, negative_slope=0.01)
        return h3

    def decode(self, Z):
        return Z @ self.decoder_weight @ Z.T

    def forward(self, X, norm_adj):
        Z = self.encode(X, norm_adj)
        A_pred = self.decode(Z)
        return A_pred, Z


# Drop edge function
def drop_edges(adj, drop_prob=0.2):
    mask = torch.rand_like(adj) > drop_prob
    adj_dropped = adj * mask
    return adj_dropped


# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate model
model = DeepGAE(in_dim=2048, h1=1024, h2=256, h3=128, dropout=0.3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Loss function with class imbalance handling
pos_weight = (A_label.numel() - A_label.sum()) / A_label.sum()
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

# Precompute upper triangle indices for edges
triu_indices = torch.triu_indices(X.shape[0], X.shape[0], offset=1)
true_edges = A_label[triu_indices[0], triu_indices[1]]

# Training
epochs = 200
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    norm_adj_dropped = drop_edges(norm_adj, drop_prob=0.2)
    A_pred, Z = model(X, norm_adj_dropped)
    pred_edges = A_pred[triu_indices[0], triu_indices[1]]

    # Loss: main + regularization
    loss = loss_fn(pred_edges, true_edges)
    reg_loss = torch.norm(Z, p=2)
    loss += 1e-4 * reg_loss

    loss.backward()
    optimizer.step()

    # Evaluation
    with torch.no_grad():
        pred_np = pred_edges.detach().cpu().numpy()
        true_np = true_edges.cpu().numpy()

        roc = roc_auc_score(true_np, pred_np)
        pr = average_precision_score(true_np, pred_np)

        precision, recall, thresholds = precision_recall_curve(true_np, pred_np)
        f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
        best_f1_idx = np.argmax(f1)
        best_f1 = f1[best_f1_idx]
        best_threshold = thresholds[best_f1_idx] if best_f1_idx < len(thresholds) else 0.5

    print(f"Epoch {epoch+1:03d} | Loss: {loss.item():.4f} | ROC-AUC: {roc:.4f} | PR-AUC: {pr:.4f} | Best-F1: {best_f1:.4f} @ Thr={best_threshold:.4f}")


Epoch 001 | Loss: 1.3879 | ROC-AUC: 0.1940 | PR-AUC: 0.0021 | Best-F1: 0.0071 @ Thr=-0.1689
Epoch 002 | Loss: 6.9265 | ROC-AUC: 0.9500 | PR-AUC: 0.0868 | Best-F1: 0.1863 @ Thr=97.7030
Epoch 003 | Loss: 963.9299 | ROC-AUC: 0.0604 | PR-AUC: 0.0018 | Best-F1: 0.0071 @ Thr=-71949.8594
Epoch 004 | Loss: 23.8222 | ROC-AUC: 0.0598 | PR-AUC: 0.0018 | Best-F1: 0.0071 @ Thr=-1753.7000
Epoch 005 | Loss: 1.2026 | ROC-AUC: 0.9430 | PR-AUC: 0.0952 | Best-F1: 0.2198 @ Thr=1.2836
Epoch 006 | Loss: 1.5627 | ROC-AUC: 0.0549 | PR-AUC: 0.0018 | Best-F1: 0.0071 @ Thr=-22.2306
Epoch 007 | Loss: 1.4736 | ROC-AUC: 0.0532 | PR-AUC: 0.0018 | Best-F1: 0.0071 @ Thr=-15.6159
Epoch 008 | Loss: 1.3187 | ROC-AUC: 0.9272 | PR-AUC: 0.0696 | Best-F1: 0.1904 @ Thr=0.3439
Epoch 009 | Loss: 1.1539 | ROC-AUC: 0.9442 | PR-AUC: 0.0675 | Best-F1: 0.1701 @ Thr=2.5184
Epoch 010 | Loss: 1.6377 | ROC-AUC: 0.9446 | PR-AUC: 0.0823 | Best-F1: 0.1980 @ Thr=16.0948
Epoch 011 | Loss: 1.1422 | ROC-AUC: 0.9454 | PR-AUC: 0.0824 | Best-F1: 

In [115]:
# After training
model.eval()
with torch.no_grad():
    A_pred_final, Z_final = model(X, norm_adj)  # [4051×4051], [4051×128]
    A_pred_np = A_pred_final.cpu().numpy()      # Final decoded matrix
    Z_np = Z_final.cpu().numpy() 

In [120]:
adj_reduced_label=Z_np
adj_reduced.shape

(4051, 128)

In [108]:
import numpy as np

def cosine_to_binary_adjacency(cos_sim_matrix: np.ndarray, percentile: float = 95.0):

    sim_no_diag = cos_sim_matrix[~np.eye(cos_sim_matrix.shape[0], dtype=bool)]
    
    threshold = np.percentile(sim_no_diag, percentile)
    
    adj_bin = (cos_sim_matrix >= threshold).astype(int)
    
    np.fill_diagonal(adj_bin, 0)
    
    return adj_bin, threshold


In [112]:
cos_sim_matrix, threshold = cosine_to_binary_adjacency(cos_sim_matrix, percentile=95.0)

print(f"Threshold used: {threshold:.4f}")
print(f"Number of edges: {np.sum(adj_bin)}")


Threshold used: 0.2957
Number of edges: 820384


In [127]:
cos_sim_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]], shape=(4051, 4051))

In [133]:
cos_sim_matrix = cos_sim_matrix + np.eye(cos_sim_matrix.shape[0])

In [134]:
def normalize_adj(adj):
    adj = adj + torch.eye(adj.shape[0]).to(device)  # Add self-loops
    deg = torch.sum(adj, dim=1)
    deg_inv_sqrt = torch.pow(deg, -0.5)
    deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0.0
    D_inv_sqrt = torch.diag(deg_inv_sqrt)
    return D_inv_sqrt @ adj @ D_inv_sqrt

cos_sim_tensor = torch.tensor(cos_sim_matrix, dtype=torch.float32).to(device)
norm_adj = normalize_adj(cos_sim_tensor)

In [135]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve
import numpy as np

# Deeper GAE model
class DeepGAE(nn.Module):
    def __init__(self, in_dim, h1, h2, h3, dropout=0.3):
        super(DeepGAE, self).__init__()
        self.W1 = nn.Parameter(torch.empty(in_dim, h1))
        self.W2 = nn.Parameter(torch.empty(h1, h2))
        self.W3 = nn.Parameter(torch.empty(h2, h3))
        self.decoder_weight = nn.Parameter(torch.empty(h3, h3))

        # Xavier initialization
        nn.init.xavier_uniform_(self.W1)
        nn.init.xavier_uniform_(self.W2)
        nn.init.xavier_uniform_(self.W3)
        nn.init.xavier_uniform_(self.decoder_weight)

        self.dropout = nn.Dropout(dropout)

    def encode(self, X, norm_adj):
        h1 = F.leaky_relu(norm_adj @ X @ self.W1, negative_slope=0.01)
        h1 = self.dropout(h1)
        h2 = F.leaky_relu(norm_adj @ h1 @ self.W2, negative_slope=0.01)
        h2 = self.dropout(h2)
        h3 = F.leaky_relu(norm_adj @ h2 @ self.W3, negative_slope=0.01)
        return h3

    def decode(self, Z):
        return Z @ self.decoder_weight @ Z.T

    def forward(self, X, norm_adj):
        Z = self.encode(X, norm_adj)
        A_pred = self.decode(Z)
        return A_pred, Z


# Drop edge function
def drop_edges(adj, drop_prob=0.2):
    mask = torch.rand_like(adj) > drop_prob
    adj_dropped = adj * mask
    return adj_dropped


# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate model
model = DeepGAE(in_dim=2048, h1=1024, h2=256, h3=128, dropout=0.3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Loss function with class imbalance handling

cos_sim_tensor = torch.tensor(cos_sim_matrix, dtype=torch.float32).to(device)
pos_weight = (cos_sim_tensor.numel() - cos_sim_tensor.sum()) / cos_sim_tensor.sum()

# pos_weight = (cos_sim_matrix.numel() - cos_sim_matrix.sum()) / cos_sim_matrix.sum()
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

# Precompute upper triangle indices for edges
triu_indices = torch.triu_indices(X.shape[0], X.shape[0], offset=1)
true_edges = cos_sim_tensor[triu_indices[0], triu_indices[1]]

# Training
epochs = 200
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    norm_adj_dropped = drop_edges(norm_adj, drop_prob=0.   )
    A_pred, Z = model(X, norm_adj_dropped)
    pred_edges = A_pred[triu_indices[0], triu_indices[1]]

    # Loss: main + regularization
    loss = loss_fn(pred_edges, true_edges)
    reg_loss = torch.norm(Z, p=2)
    loss += 1e-4 * reg_loss

    loss.backward()
    optimizer.step()

    with torch.no_grad():
        pred_np = pred_edges.detach().cpu().numpy()
        true_np = true_edges.cpu().numpy()

        roc = roc_auc_score(true_np, pred_np)
        pr = average_precision_score(true_np, pred_np)

        precision, recall, thresholds = precision_recall_curve(true_np, pred_np)
        f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
        best_f1_idx = np.argmax(f1)
        best_f1 = f1[best_f1_idx]
        best_threshold = thresholds[best_f1_idx] if best_f1_idx < len(thresholds) else 0.5

    print(f"Epoch {epoch+1:03d} | Loss: {loss.item():.4f} | ROC-AUC: {roc:.4f} | PR-AUC: {pr:.4f} | Best-F1: {best_f1:.4f} @ Thr={best_threshold:.4f}")


Epoch 001 | Loss: 1.3063 | ROC-AUC: 0.7710 | PR-AUC: 0.2047 | Best-F1: 0.2704 @ Thr=0.0705
Epoch 002 | Loss: 164.3316 | ROC-AUC: 0.7841 | PR-AUC: 0.2239 | Best-F1: 0.2913 @ Thr=340.1270
Epoch 003 | Loss: 3.0357 | ROC-AUC: 0.2777 | PR-AUC: 0.0313 | Best-F1: 0.0952 @ Thr=-9.0135
Epoch 004 | Loss: 213.8496 | ROC-AUC: 0.7731 | PR-AUC: 0.1995 | Best-F1: 0.2601 @ Thr=420.9375
Epoch 005 | Loss: 22.2882 | ROC-AUC: 0.2372 | PR-AUC: 0.0298 | Best-F1: 0.0952 @ Thr=-72.4417
Epoch 006 | Loss: 5.4183 | ROC-AUC: 0.7291 | PR-AUC: 0.1140 | Best-F1: 0.1926 @ Thr=8.7372
Epoch 007 | Loss: 2.5262 | ROC-AUC: 0.2516 | PR-AUC: 0.0303 | Best-F1: 0.0952 @ Thr=-7.0049
Epoch 008 | Loss: 1.3499 | ROC-AUC: 0.7734 | PR-AUC: 0.2128 | Best-F1: 0.2777 @ Thr=1.3286
Epoch 009 | Loss: 1.9984 | ROC-AUC: 0.3203 | PR-AUC: 0.0332 | Best-F1: 0.0955 @ Thr=-3.5004
Epoch 010 | Loss: 7.7698 | ROC-AUC: 0.6696 | PR-AUC: 0.0850 | Best-F1: 0.1506 @ Thr=8.5566
Epoch 011 | Loss: 1.4005 | ROC-AUC: 0.7669 | PR-AUC: 0.1843 | Best-F1: 0.252

In [136]:
model.eval()
with torch.no_grad():
    A_pred_final, Z_final = model(X, norm_adj)  # [4051×4051], [4051×128]
    A_pred_np = A_pred_final.cpu().numpy()      # Final decoded matrix
    Z_np = Z_final.cpu().numpy() 

In [137]:
cosine =Z_np

In [138]:
cosine.shape

(4051, 128)

In [140]:
np.save('cosine.npy', cosine)  # Saves to 'my_array.npy'

np.save('adj_label.npy', adj_reduced)  # Saves to 'my_array.npy'


In [35]:
cosine_label = np.load('cosine.npy')
adj_label = np.load('adj_label.npy')

In [36]:
cosine_label.shape

(4051, 128)

In [37]:
adj_label.shape

(4051, 128)

In [38]:
cell_feature.head()

,cell_line_name,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,...,ENSG00000273238,ENSG00000273259,ENSG00000273269,ENSG00000273274,ENSG00000273291,ENSG00000273294,ENSG00000273313,ENSG00000273331,ENSG00000273345,ENSG00000273398
0,A2058,15.67,0.00,97.03,5.11,23.03,0.02,7.43,94.70,14.35,...,0.00,0.28,0.0,0.08,0.0,0.87,0.00,0.00,0.18,0.06
1,A2780,94.32,2.53,119.23,5.92,21.57,0.03,248.72,19.78,12.86,...,0.06,0.00,0.0,0.13,0.0,0.00,0.02,0.00,0.22,0.03
3,A427,19.18,0.00,95.22,2.73,11.14,0.01,0.13,45.33,10.43,...,0.00,0.00,0.0,0.16,0.0,0.13,0.04,0.02,0.06,0.05
20,NCI-H460,23.21,0.00,82.35,2.74,19.65,0.02,12.89,43.91,65.63,...,0.00,0.00,0.0,0.00,0.0,0.06,0.23,0.23,0.06,0.07
26,RKO,11.89,0.02,153.59,1.43,12.71,0.00,0.13,56.71,15.72,...,0.00,0.00,0.0,0.00,0.0,0.00,6.03,0.49,0.49,0.00


In [39]:
drug_interaction.head()

,drug_row_id,drug_col_id,cell_line_name,synergistic_status
0,0,1,A2058,1
1,0,1,A2058,1
2,0,1,A2058,1
3,0,1,A2058,0
4,0,2,A2058,1


In [63]:
import torch.nn as nn

class GeneExpressionAutoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(19225, 4096),
            nn.ReLU(),
            nn.Linear(4096, 1024),
            nn.ReLU(),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Linear(256, 128)
        )
        self.decoder = nn.Sequential(
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 1024),
            nn.ReLU(),
            nn.Linear(1024, 4096),
            nn.ReLU(),
            nn.Linear(4096, 19225)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded, encoded

model = GeneExpressionAutoencoder()


In [64]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset

# Assuming your dataframe is called df
cell_names = cell_feature.iloc[:, 0]  # first column, cell line names
gene_features = cell_feature.iloc[:, 1:].values  # numeric matrix (717, 19226)


In [65]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
gene_features_scaled = scaler.fit_transform(gene_features)


In [66]:
X = torch.tensor(gene_features_scaled, dtype=torch.float32)

dataset = TensorDataset(X)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [67]:
X.shape

torch.Size([717, 19225])

In [68]:
cell_feature

,cell_line_name,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,...,ENSG00000273238,ENSG00000273259,ENSG00000273269,ENSG00000273274,ENSG00000273291,ENSG00000273294,ENSG00000273313,ENSG00000273331,ENSG00000273345,ENSG00000273398
0,A2058,15.67,0.00,97.03,5.11,23.03,0.02,7.43,94.70,14.35,...,0.00,0.28,0.0,0.08,0.00,0.87,0.00,0.00,0.18,0.06
1,A2780,94.32,2.53,119.23,5.92,21.57,0.03,248.72,19.78,12.86,...,0.06,0.00,0.0,0.13,0.00,0.00,0.02,0.00,0.22,0.03
3,A427,19.18,0.00,95.22,2.73,11.14,0.01,0.13,45.33,10.43,...,0.00,0.00,0.0,0.16,0.00,0.13,0.04,0.02,0.06,0.05
20,NCI-H460,23.21,0.00,82.35,2.74,19.65,0.02,12.89,43.91,65.63,...,0.00,0.00,0.0,0.00,0.00,0.06,0.23,0.23,0.06,0.07
26,RKO,11.89,0.02,153.59,1.43,12.71,0.00,0.13,56.71,15.72,...,0.00,0.00,0.0,0.00,0.00,0.00,6.03,0.49,0.49,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1348,SNU-503,28.61,0.00,70.73,4.97,9.07,0.00,0.20,6.87,218.71,...,0.00,0.00,0.0,0.04,0.05,0.20,0.65,0.23,0.19,0.25
1349,SNU-869,29.30,0.00,56.78,5.69,8.71,0.11,1.09,76.30,64.22,...,0.00,2.15,0.0,0.04,0.00,0.04,2.56,0.06,0.06,0.07
1350,UM-UC-1,77.94,0.00,138.02,3.08,4.20,0.03,6.40,43.69,33.56,...,0.00,0.00,0.0,0.10,0.00,0.00,0.00,0.03,0.20,0.00
1352,HuG1-N,0.06,0.00,79.03,4.48,11.59,0.05,0.04,87.70,134.13,...,0.00,0.00,0.0,0.01,0.00,0.14,0.08,0.00,0.05,0.07


In [69]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        inputs = batch[0]
        optimizer.zero_grad()
        outputs, _ = model(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * inputs.size(0)
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataset):.4f}")


Epoch 1, Loss: 0.9434
Epoch 2, Loss: 0.8812
Epoch 3, Loss: 0.8608
Epoch 4, Loss: 0.8496
Epoch 5, Loss: 0.8313
Epoch 6, Loss: 0.8330
Epoch 7, Loss: 0.8227
Epoch 8, Loss: 0.7950
Epoch 9, Loss: 0.7891
Epoch 10, Loss: 0.7808
Epoch 11, Loss: 0.7635
Epoch 12, Loss: 0.7549
Epoch 13, Loss: 0.7470
Epoch 14, Loss: 0.7523
Epoch 15, Loss: 0.7500
Epoch 16, Loss: 0.7456
Epoch 17, Loss: 0.7385
Epoch 18, Loss: 0.7274
Epoch 19, Loss: 0.7119
Epoch 20, Loss: 0.7226
Epoch 21, Loss: 0.7056
Epoch 22, Loss: 0.7014
Epoch 23, Loss: 0.6938
Epoch 24, Loss: 0.6878
Epoch 25, Loss: 0.7427
Epoch 26, Loss: 0.6904
Epoch 27, Loss: 0.6796
Epoch 28, Loss: 0.6770
Epoch 29, Loss: 0.6691
Epoch 30, Loss: 0.6565
Epoch 31, Loss: 0.6448
Epoch 32, Loss: 0.6577
Epoch 33, Loss: 0.6441
Epoch 34, Loss: 0.6283
Epoch 35, Loss: 0.6227
Epoch 36, Loss: 0.6103
Epoch 37, Loss: 0.6049
Epoch 38, Loss: 0.5952
Epoch 39, Loss: 0.5941
Epoch 40, Loss: 0.5837
Epoch 41, Loss: 0.5693
Epoch 42, Loss: 0.5651
Epoch 43, Loss: 0.5527
Epoch 44, Loss: 0.55

In [70]:
model.eval()
with torch.no_grad():
    X_tensor = torch.tensor(gene_features_scaled, dtype=torch.float32)
    _, encoded_features = model(X_tensor)



In [71]:
encoded_np = encoded_features.cpu().numpy()  # if on GPU, else just .numpy()

# Extract cell line names (assuming your original df is called cell_feature)
cell_names = cell_feature.iloc[:, 0].reset_index(drop=True)

# Create new DataFrame
encoded_df = pd.DataFrame(encoded_np, columns=[f'encoded_feat_{i}' for i in range(encoded_np.shape[1])])

# Combine cell names with encoded features
final_df = pd.concat([cell_names, encoded_df], axis=1)

In [86]:
import numpy as np
import pandas as pd



cell_feat_map = final_df.set_index(final_df.columns[0])  # index on cell_line_name, columns are features

# Prepare lists to collect features
combined_features = []
labels = []
skipped_rows = 0

for idx, row in drug_interaction.iterrows():
    drug_row_idx = row['drug_row_id']
    drug_col_idx = row['drug_col_id']
    cell_name = row['cell_line_name']
    label = row['synergistic_status']

    # Check if cell name exists in final_df
    if cell_name not in cell_feat_map.index:
        skipped_rows += 1
        continue  # skip this row

    # Extract drug features
    try:
        drug_row_feat = cosine_label[drug_row_idx]
        drug_col_feat = cosine_label[drug_col_idx]
    except IndexError:
        skipped_rows += 1
        continue  # skip if drug id is out of bounds

    # Extract cell features
    cell_feat = cell_feat_map.loc[cell_name].values  # shape (128,)

    # Combine all
    combined = np.concatenate([drug_row_feat, drug_col_feat, cell_feat])  # (384,)
    combined_features.append(combined)
    labels.append(label)

# Convert to final DataFrame
combined_features = np.array(combined_features)
labels = np.array(labels)

feature_cols = [f'drug_row_feat_{i}' for i in range(128)] + \
               [f'drug_col_feat_{i}' for i in range(128)] + \
               [f'cell_feat_{i}' for i in range(128)]

final_dataset = pd.DataFrame(combined_features, columns=feature_cols)
final_dataset['synergistic_status'] = labels

print(f"Final dataset shape: {final_dataset.shape}")
print(f"Skipped rows due to missing cell line or drug index: {skipped_rows}")
print(final_dataset.head())

Final dataset shape: (166227, 385)
Skipped rows due to missing cell line or drug index: 298517
   drug_row_feat_0  drug_row_feat_1  drug_row_feat_2  drug_row_feat_3  \
0        -0.008872        -0.003799        -0.011806        -0.002723   
1        -0.008872        -0.003799        -0.011806        -0.002723   
2        -0.008872        -0.003799        -0.011806        -0.002723   
3        -0.008872        -0.003799        -0.011806        -0.002723   
4        -0.008872        -0.003799        -0.011806        -0.002723   

   drug_row_feat_4  drug_row_feat_5  drug_row_feat_6  drug_row_feat_7  \
0        -0.015133         0.013551        -0.010493         -0.00656   
1        -0.015133         0.013551        -0.010493         -0.00656   
2        -0.015133         0.013551        -0.010493         -0.00656   
3        -0.015133         0.013551        -0.010493         -0.00656   
4        -0.015133         0.013551        -0.010493         -0.00656   

   drug_row_feat_8  drug_ro

In [88]:
final_dataset.shape

(166227, 385)

In [92]:
final_dataset.head()

,drug_row_feat_0,drug_row_feat_1,drug_row_feat_2,drug_row_feat_3,drug_row_feat_4,drug_row_feat_5,drug_row_feat_6,drug_row_feat_7,drug_row_feat_8,drug_row_feat_9,...,cell_feat_119,cell_feat_120,cell_feat_121,cell_feat_122,cell_feat_123,cell_feat_124,cell_feat_125,cell_feat_126,cell_feat_127,synergistic_status
0,-0.008872,-0.003799,-0.011806,-0.002723,-0.015133,0.013551,-0.010493,-0.00656,-0.004526,-0.006435,...,5.578652,-2.544904,-11.148895,2.738184,-11.305983,9.668481,1.015076,-3.399888,-11.600459,1
1,-0.008872,-0.003799,-0.011806,-0.002723,-0.015133,0.013551,-0.010493,-0.00656,-0.004526,-0.006435,...,5.578652,-2.544904,-11.148895,2.738184,-11.305983,9.668481,1.015076,-3.399888,-11.600459,1
2,-0.008872,-0.003799,-0.011806,-0.002723,-0.015133,0.013551,-0.010493,-0.00656,-0.004526,-0.006435,...,5.578652,-2.544904,-11.148895,2.738184,-11.305983,9.668481,1.015076,-3.399888,-11.600459,1
3,-0.008872,-0.003799,-0.011806,-0.002723,-0.015133,0.013551,-0.010493,-0.00656,-0.004526,-0.006435,...,5.578652,-2.544904,-11.148895,2.738184,-11.305983,9.668481,1.015076,-3.399888,-11.600459,0
4,-0.008872,-0.003799,-0.011806,-0.002723,-0.015133,0.013551,-0.010493,-0.00656,-0.004526,-0.006435,...,5.578652,-2.544904,-11.148895,2.738184,-11.305983,9.668481,1.015076,-3.399888,-11.600459,1


In [96]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score, roc_curve, precision_recall_curve
import matplotlib.pyplot as plt
X = final_dataset.iloc[:, :-1].values  # all features
y = final_dataset.iloc[:, -1].values

In [97]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [98]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

In [99]:
class MLP(nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.net(x)

model = MLP(X_train.shape[1])


In [104]:
# Loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Tracking
train_losses = []
roc_aucs = []
pr_aucs = []

# Training loop
for epoch in range(1000):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    
    train_losses.append(loss.item())
    
    # Evaluation (on test set)
    model.eval()
    with torch.no_grad():
        preds = model(X_test)
        preds_np = preds.detach().numpy()
        y_test_np = y_test.detach().numpy()
        
        roc_auc = roc_auc_score(y_test_np, preds_np)
        pr_auc = average_precision_score(y_test_np, preds_np)
        
        roc_aucs.append(roc_auc)
        pr_aucs.append(pr_auc)

    print(f"Epoch {epoch+1:02d}: Loss={loss.item():.4f}, ROC-AUC={roc_auc:.4f}, PR-AUC={pr_auc:.4f}")


Epoch 01: Loss=0.6386, ROC-AUC=0.6242, PR-AUC=0.5682
Epoch 02: Loss=0.8086, ROC-AUC=0.6467, PR-AUC=0.5878
Epoch 03: Loss=0.6510, ROC-AUC=0.6410, PR-AUC=0.5846
Epoch 04: Loss=0.6855, ROC-AUC=0.6343, PR-AUC=0.5786
Epoch 05: Loss=0.7218, ROC-AUC=0.6395, PR-AUC=0.5832
Epoch 06: Loss=0.6895, ROC-AUC=0.6427, PR-AUC=0.5865
Epoch 07: Loss=0.6534, ROC-AUC=0.6397, PR-AUC=0.5840
Epoch 08: Loss=0.6434, ROC-AUC=0.6347, PR-AUC=0.5788
Epoch 09: Loss=0.6529, ROC-AUC=0.6323, PR-AUC=0.5764
Epoch 10: Loss=0.6638, ROC-AUC=0.6337, PR-AUC=0.5771
Epoch 11: Loss=0.6660, ROC-AUC=0.6365, PR-AUC=0.5794
Epoch 12: Loss=0.6599, ROC-AUC=0.6379, PR-AUC=0.5815
Epoch 13: Loss=0.6514, ROC-AUC=0.6366, PR-AUC=0.5814
Epoch 14: Loss=0.6461, ROC-AUC=0.6343, PR-AUC=0.5796
Epoch 15: Loss=0.6456, ROC-AUC=0.6332, PR-AUC=0.5783
Epoch 16: Loss=0.6483, ROC-AUC=0.6339, PR-AUC=0.5786
Epoch 17: Loss=0.6510, ROC-AUC=0.6357, PR-AUC=0.5801
Epoch 18: Loss=0.6518, ROC-AUC=0.6377, PR-AUC=0.5824
Epoch 19: Loss=0.6506, ROC-AUC=0.6391, PR-AUC=

In [101]:
# Binary predictions for report
preds_bin = (preds_np >= 0.5).astype(int)

print("\nClassification Report:")
print(classification_report(y_test_np, preds_bin, digits=4))



Classification Report:
              precision    recall  f1-score   support

         0.0     0.6210    0.8266    0.7092     19236
         1.0     0.5635    0.3074    0.3978     14010

    accuracy                         0.6078     33246
   macro avg     0.5923    0.5670    0.5535     33246
weighted avg     0.5968    0.6078    0.5780     33246



In [103]:
# Binary predictions for report
preds_bin = (preds_np >= 0.5).astype(int)

print("\nClassification Report:")
print(classification_report(y_test_np, preds_bin, digits=4))



Classification Report:
              precision    recall  f1-score   support

         0.0     0.6342    0.8610    0.7304     19236
         1.0     0.6251    0.3182    0.4217     14010

    accuracy                         0.6323     33246
   macro avg     0.6296    0.5896    0.5761     33246
weighted avg     0.6304    0.6323    0.6003     33246



In [105]:
# Binary predictions for report
preds_bin = (preds_np >= 0.5).astype(int)

print("\nClassification Report:")
print(classification_report(y_test_np, preds_bin, digits=4))



Classification Report:
              precision    recall  f1-score   support

         0.0     0.6678    0.7750    0.7174     19236
         1.0     0.6037    0.4707    0.5290     14010

    accuracy                         0.6468     33246
   macro avg     0.6358    0.6228    0.6232     33246
weighted avg     0.6408    0.6468    0.6380     33246

